In [20]:
import os; os.environ['OPENBLAS_NUM_THREADS']='1'
import numpy as np
import pandas as pd
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k, ndcg_at_k
from sklearn.model_selection import StratifiedGroupKFold
import gc
import collections
from tqdm import tqdm

In [21]:
# train упорядочен хронологически
# timespent: время залипания юзера на айтем в минутах (от 0 до 60)
# reaction: (1) - лайк, (-1) - дизлайк
train = pd.read_parquet('misha_data/train_2degree_value.parquet.gzip')
# train = train.reset_index().rename(columns={"index": "time"}) # set discrete time
# train['user_time'] = train.groupby('user_id').cumcount() # set user-wise discrete time

# train = train[train.timespent > 0] # sample filters
# train = train[train.reaction != -1]

ALL_USERS = train['user_id'].unique().tolist()
ALL_ITEMS = train['item_id'].unique().tolist()

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

train['user_id_new'] = train['user_id'].map(user_map)
train['item_id_new'] = train['item_id'].map(item_map)

print("len rows: ", train.shape[0])
print("Unique users: ", train.user_id.nunique())
print("Unique items: ", train.item_id.nunique())
train.head()

In [24]:
train['value'] = 1 + train.user_time_norm_nlinear * 40 * train.timespent # data for impact in als
train.head()

,time,user_id,item_id,timespent,reaction,user_time,user_id_new,item_id_new,user_time_norm_nlinear,value
0,0,707536,67950,0,0,0,0,0,0.000000,1.0
1,1,707536,151002,0,0,1,0,1,0.000008,1.0
2,2,707536,134736,0,0,2,0,2,0.000033,1.0
3,3,707536,196151,0,0,3,0,3,0.000074,1.0
4,4,707536,94182,0,0,4,0,4,0.000132,1.0


In [25]:
# в items_meta для каждого item_id его автор и эмбеддинг содержания
items_meta = pd.read_parquet('items_meta.parquet.gzip')
print("len rows: ", items_meta.shape[0])
print("Unique items: ", items_meta.item_id.nunique())
print("Unique writers: ", items_meta.source_id.nunique())
print("emb shape: ", items_meta.iloc[0].embeddings.shape)
items_meta.head()

len rows:  227606
Unique items:  227606
Unique writers:  24438
emb shape:  (312,)


,item_id,source_id,embeddings
0,0,7340,"[0.10458118, 0.047880154, 0.030944156, -0.0351..."
1,1,6284,"[0.035625108, -0.039264094, -0.03310334, -0.04..."
2,2,12766,"[0.08418761, 0.006732465, -0.0037112322, -0.02..."
3,3,14734,"[0.049901545, 0.039079394, -0.03890682, -0.053..."
4,4,22557,"[0.09303163, 0.023448057, 0.0029488814, -0.017..."


In [26]:
# candidates содержит item_id свежих кандидатов из которых нужно будет предсказать на тесте
candidates_df = pd.read_parquet('fresh_candidates.parquet.gzip')
print("len rows: ", candidates_df.shape[0])
print("Items for predict: ", candidates_df.item_id.nunique())
candidates_df.head()

len rows:  100000
Items for predict:  100000


,item_id
0,0
1,2
2,5
3,6
4,7


In [27]:
# find author's items
author_items = collections.defaultdict(list)
max_len = 0

tmp = items_meta[items_meta.item_id.isin(candidates_df.item_id.values)]

for id, row in tqdm(tmp.iterrows(), total=tmp.shape[0]):
    author_items[row.source_id].append(row.item_id)
    l = len(author_items[row.source_id])
    if l > max_len:
        max_len = l

del tmp
gc.collect()
print(max_len)

100%|██████████| 100000/100000 [00:07<00:00, 13397.94it/s]


619

In [28]:
test = pd.read_parquet('test.parquet.gzip')
print("len rows: ", test.shape[0])
print("Test users: ", test.user_id.nunique())
test.head()

len rows:  200000
Test users:  200000


,user_id
0,7
1,8
2,9
3,11
4,18


# Train/Val Split

In [10]:
%%time
val_size = 0.2

val_df = train[train.time >= train.quantile(1 - val_size)["time"]]
train_df = train[train.time < train.quantile(1 - val_size)["time"]]

CPU times: user 9.95 s, sys: 3.05 s, total: 13 s
Wall time: 13 s


In [11]:
print("Val size users: ", len(set(val_df.user_id.unique())))
print("Train size users: ", len(set(train_df.user_id.unique())))
print("Intersection size users: ", len(set(val_df.user_id.unique()).intersection(set(train_df.user_id.unique()))))
# Val size users:  685906
# Train size users:  797996
# Intersection size users:  683719

Val size users:  786401
Train size users:  991224
Intersection size users:  777442


In [29]:
def to_user_item_coo(df):
    """ Turn a dataframe with transactions into a COO sparse items x users matrix"""
    row = df['user_id_new'].values
    col = df['item_id_new'].values
    data = df['value'].values
    coo = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
    return coo


def split_data(df, validation_size=0.15):
    """ Split a pandas dataframe into training and validation data, using <<validation_days>>
    """
    validation_cut = df.shape[0] * (1 - validation_size)

    df_train = df[df['time'] < validation_cut]
    df_val = df[df['time'] >= validation_cut]
    return df_train, df_val

def get_val_matrices(df, validation_size=0.15):
    """ Split into training and validation and create various matrices
        
        Returns a dictionary with the following keys:
            coo_train: training data in COO sparse format and as (users x items)
            csr_train: training data in CSR sparse format and as (users x items)
            csr_val:  validation data in CSR sparse format and as (users x items)
    
    """
    df_train, df_val = split_data(df, validation_size=validation_size)
    coo_train = to_user_item_coo(df_train)
    coo_val = to_user_item_coo(df_val)

    csr_train = coo_train.tocsr()
    csr_val = coo_val.tocsr()
    
    return {'coo_train': coo_train,
            'csr_train': csr_train,
            'csr_val': csr_val
          }


def validate(matrices, factors=200, iterations=20, regularization=0.01, alpha=40, show_progress=True):
    """ Train an ALS model with <<factors>> (embeddings dimension) 
    for <<iterations>> over matrices and validate with MAP@12
    """
    coo_train, csr_train, csr_val = matrices['coo_train'], matrices['csr_train'], matrices['csr_val']
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42,
                                                 alpha=alpha,
                                                 use_gpu=True)
    model.fit(csr_train, show_progress=show_progress)
    
    # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
    # TODO: change MAP@12 to a library that allows repeated items in prediction
    ndcg20 = ndcg_at_k(model, csr_train, csr_val, K=20, show_progress=show_progress, num_threads=4)
    map20 = mean_average_precision_at_k(model, csr_train, csr_val, K=20, show_progress=show_progress, num_threads=4)
    print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} - Alpha: {alpha} ==> NDCG@20: {ndcg20:6.5f} ==> MAP@20: {map20:6.5f}")
    return ndcg20, map20

In [30]:
matrices = get_val_matrices(train) # get train/val matrices

In [32]:
%%time
# params tuning
best_ndcg20 = 0
best_map20 = 0
for iterations in range(7, 16):
    for factors in [300, 350, 400, 450]:
        for regularization in [0.001]:
            for alpha in [1]:
                ndcg20, map20 = validate(matrices, factors, iterations, regularization, alpha, show_progress=False)
                if ndcg20 > best_ndcg20:
                    best_ndcg20 = ndcg20
                    best_params_ndcg = {'factors': factors, 'iterations': iterations, 'regularization': regularization, 'alpha': alpha}
                    print(f"Best NDCG@20 found. Updating: {best_params_ndcg}")
                if map20 > best_map20:
                    best_map20 = map20
                    best_params_map = {'factors': factors, 'iterations': iterations, 'regularization': regularization, 'alpha': alpha}
                    print(f"Best MAP@20 found. Updating: {best_params_map}")

Factors: 300 - Iterations:  7 - Regularization: 0.001 - Alpha: 1 ==> NDCG@20: 0.08626 ==> MAP@20: 0.03231
Best NDCG@20 found. Updating: {'factors': 300, 'iterations': 7, 'regularization': 0.001, 'alpha': 1}
Best MAP@20 found. Updating: {'factors': 300, 'iterations': 7, 'regularization': 0.001, 'alpha': 1}
Factors: 350 - Iterations:  7 - Regularization: 0.001 - Alpha: 1 ==> NDCG@20: 0.08657 ==> MAP@20: 0.03248
Best NDCG@20 found. Updating: {'factors': 350, 'iterations': 7, 'regularization': 0.001, 'alpha': 1}
Best MAP@20 found. Updating: {'factors': 350, 'iterations': 7, 'regularization': 0.001, 'alpha': 1}
Factors: 400 - Iterations:  7 - Regularization: 0.001 - Alpha: 1 ==> NDCG@20: 0.08676 ==> MAP@20: 0.03256
Best NDCG@20 found. Updating: {'factors': 400, 'iterations': 7, 'regularization': 0.001, 'alpha': 1}
Best MAP@20 found. Updating: {'factors': 400, 'iterations': 7, 'regularization': 0.001, 'alpha': 1}
Factors: 450 - Iterations:  7 - Regularization: 0.001 - Alpha: 1 ==> NDCG@20: 0

KeyboardInterrupt: 

## Train on all data

In [33]:
coo_train = to_user_item_coo(train)
csr_train = coo_train.tocsr()

In [44]:
def train(coo_train, factors=200, iterations=15, regularization=0.01, alpha=40, show_progress=True):
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=1945,
                                                 alpha=alpha, 
                                                 use_gpu=True
                                                )
    model.fit(coo_train, show_progress=False)
    return model

In [45]:
best_params = {'factors': 400, 'iterations': 9, 'regularization': 0.001, 'alpha': 1}

In [46]:
model = train(csr_train, **best_params)

In [47]:
items_for_pred = []
for item in candidates_df.item_id.tolist():
    try:
        items_for_pred.append(item_map[item])
    except:
        continue

print("warm items: ", len(items_for_pred))

warm items:  100000


In [48]:
warm_users = []
cold_users = []
for user in test.user_id.tolist():
    try:
        warm_users.append(user_map[user])
    except:
        cold_users.append(user)

print("warm users: ", len(warm_users))
print("cold users: ", len(cold_users))

warm users:  200000
cold users:  0


In [49]:
def submit(model, csr_train, submission_name="sub.parquet.gzip"):
    preds = []
    most_populars = [4628, 103927, 146586, 18584, 75560, 44269, 58977, 227420, 130953, 11244, 130122, 173607, 121430, 195239, 73059, 52801, 105708, 224095, 55854, 24951]

    ids, scores = model.recommend(
        warm_users, 
        csr_train[warm_users], 
        N=20+max_len, 
        filter_already_liked_items=True, 
        items=items_for_pred)
    
    for i, userid in enumerate(warm_users):
        user_id = user_ids[userid]
        user_items = ids[i]
        article_ids = [item_ids[item_id] for item_id in user_items]
        filter_items = author_items[user_id]
        article_ids = [item for item in article_ids if item not in filter_items]
        preds.append((user_id, article_ids[:20]))
        
    for userid in cold_users:
        preds.append((userid, most_views))            

    df_preds = pd.DataFrame(preds, columns=['user_id', 'predictions'])
    
    df_preds.to_parquet(submission_name, compression='gzip')
    
    display(df_preds.head())
    print(df_preds.shape)
    
    return df_preds

In [50]:
%%time
df_preds = submit(model, csr_train);

,user_id,predictions
0,7,"[115127, 77577, 63017, 221001, 162251, 194570,..."
1,8,"[142183, 97249, 44222, 216317, 134460, 163702,..."
2,9,"[32474, 63495, 227299, 105130, 61240, 67723, 2..."
3,11,"[143520, 75961, 180137, 63388, 177667, 211646,..."
4,18,"[190377, 120767, 97006, 129830, 189621, 212829..."


(200000, 2)
CPU times: user 1min 11s, sys: 1.73 s, total: 1min 13s
Wall time: 1min 13s


## Addons, like ranking/reranking

In [17]:
tmp = df_preds.explode('predictions').merge(mean_timespent, left_on='predictions', right_on='item_id').drop(columns=['item_id']).sort_values(by=['user_id', 'timespent'], ascending=False).drop(columns=['timespent']).groupby('user_id').agg(list).reset_index()

In [18]:
tmp.to_parquet('implicit_sub_sort_mean_all.parquet.gzip', compression='gzip')

In [19]:
tmp

,user_id,predictions
0,7,"[2216, 194570, 101739, 12697, 182908, 80394, 9..."
1,8,"[62274, 49912, 190438, 40628, 186181, 53828, 9..."
2,9,"[108363, 63495, 96717, 149513, 113616, 29342, ..."
3,11,"[142181, 4305, 152520, 191744, 148826, 31753, ..."
4,18,"[169890, 2216, 155056, 34251, 182733, 206293, ..."
...,...,...
199995,1000160,"[55352, 25218, 52187, 169617, 120338, 220549, ..."
199996,1000165,"[11231, 120027, 78285, 14866, 165314, 41068, 2..."
199997,1000166,"[96284, 126127, 125070, 182908, 135532, 146558..."
199998,1000168,"[32348, 62046, 55382, 179166, 98609, 119088, 5..."
